In [1]:
try:
    from configuracoes_notebooks import set_proj_dir
    set_proj_dir()
except:
    from teste import diretorio_teste
    diretorio_teste()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\Olívia\Desktop\batatinhas\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Ocorrências de Inundações / Área do Distritos

In [3]:
data_path = get_data_diretorio()
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências
Este notebook é dependente do arquivo resultante do notebok '.alaga_por_dist'

In [4]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'areas_alagamento',
        'qt_ocorrencias_alaga_distrito.parquet'
    )
)

In [5]:
gdf_distritos.sample(3)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_alaga_ocor
81,3550308,São Paulo,355030882,TUCURUVI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,9.448038,99559,46695,"POLYGON ((337127.372 7403854.043, 337182.947 7...",0.0
77,3550308,São Paulo,355030878,SE,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,2.180669,23832,13885,"POLYGON ((333930.933 7394149.741, 333884.447 7...",0.0
71,3550308,São Paulo,355030872,SAO LUCAS,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,9.665411,138038,60157,"POLYGON ((342744.375 7388097.162, 342725.06 73...",1.0


# Cálculo

In [6]:
gdf_distritos['inund_por_area'] = (
    gdf_distritos['qt_alaga_ocor'] / gdf_distritos['geometry'].area
)

# Padronização de Entrega

In [7]:
gdf_distritos.columns


Index(['CD_MUN', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_RGINT', 'NM_RGINT',
       'CD_RGI', 'NM_RGI', 'CD_CONCURB', 'NM_CONCURB', 'AREA_KM2', 'total_pop',
       'total_dom', 'geometry', 'qt_alaga_ocor', 'inund_por_area'],
      dtype='object')

In [8]:
drop_cols={
    'CD_MUN',
    'NM_MUN',
    'CD_RGINT',
    'NM_RGINT',
    'CD_RGI',
    'NM_RGI',
    'CD_CONCURB',
    'NM_CONCURB',
    'AREA_KM2',
    'geometry'
}

gdf_distritos.drop(
    columns=drop_cols,
    axis=1,
    inplace=True
)

In [9]:
gdf_distritos.sample(2)

,CD_DIST,NM_DIST,total_pop,total_dom,qt_alaga_ocor,inund_por_area
57,355030858,PEDREIRA,163586,64002,1.0,5.410681e-08
64,355030865,RAPOSO TAVARES,117738,49006,5.0,4.110302e-07


In [10]:
gdf_distritos.set_index('NM_DIST', inplace=True)

In [11]:
gdf_distritos.sample(2)

,CD_DIST,total_pop,total_dom,qt_alaga_ocor,inund_por_area
NM_DIST,,,,,
GRAJAU,355030830,384873,154205,4.0,4.321050e-08
MOEMA,355030832,81899,46035,1.0,1.098326e-07


# Salvar GDF

In [12]:
save_parquet_excel(
    gdf_distritos,
    fname='alagamento_por_area_distrito',
    data_path = assets_path,
    data_subpath = 'areas_alagamento'
)